# Taxi Fares Kaggle Challenge

This project is to forecast the taxi fares in New York city according to a set of data already provided. This is a Kaggle Challenge.


## Load the Data

Read the data from all the CSV files and append the dataset.


In [1]:
import pandas as pd
import os, os.path

path = 'C:/Users/Mat/Documents/GitHub/TaxiFares/Datasets/'

def find_csv_filenames(path, suffix=".csv"):
    filenames = os.listdir(path)
    return [f for f in filenames if f.endswith(suffix)]

filenames = find_csv_filenames(path)

n = 0 # for testing only
df = pd.DataFrame()
for filename in filenames:
    if n < 1: # for testing only
        df = df.append(pd.read_csv(path + filename, index_col = 0))
        print("The file " + filename + " is read.")
        n += 1 # for testing only
    else: # for testing only
        break # for testing only

df.head()

The file data_0_500000.csv is read.


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.000000100,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.000000200,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.000000490,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.000000100,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1


In [17]:
print("Total number of records read: {:,}".format(len(df)))

Total number of records: 500,000


## Split pickup_datetime into multiple variables

Add variables for year, month, date, weekday, hour and holiday.

In [32]:
# Convert np arrays in datetime format
df['key'] = pd.to_datetime(df.iloc[:]['key'])
df['pickup_datetime'] = pd.to_datetime(df.iloc[:]['pickup_datetime'])

In [ ]:
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar

cal = USFederalHolidayCalendar()
calHoliday = cal.holidays(start=df['pickup_datetime'].min(), end=df['pickup_datetime'].max())
print(calHoliday)
# Create new variables
year = []
month = []
day = []
weekday = []
hour = []
holiday = []
for i in range(0, len(df)):
    y = df.iloc[i]['pickup_datetime'].year
    year.append(y)
    m = df.iloc[i]['pickup_datetime'].month
    month.append(m)
    d = df.iloc[i]['pickup_datetime'].day
    day.append(d)
    weekday.append(df.iloc[i]['pickup_datetime'].weekday())
    hour.append(df.iloc[i]['pickup_datetime'].hour)
    if datetime.date(y, m, d) in calHoliday:
        isHoliday = True
    else: 
        isHoliday = False
    holiday.append(isHoliday)

# Add new data to dataframe
df['year'] = pd.Series(year, index=df.index)
df['month'] = pd.Series(month, index=df.index)
df['day'] = pd.Series(day, index=df.index)
df['weekday'] = pd.Series(weekday, index=df.index)
df['hour'] = pd.Series(hour, index=df.index)
df['holiday'] = pd.Series(holiday, index=df.index)

df.head()

In [ ]:
df['holiday'].sum(axis=1)

## Dummy variables

Create binary dummy variables for year, month, day, weekday, timeofday and holidays.

In [ ]:
dummy_fields = ['year', 'month', 'day', 'weekday', 'hour']
for each in dummy_fields:
    dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
    df = pd.concat([df, dummies], axis=1)

fields_to_drop = ['key', 'pickup_datetime']
data = df.drop(fields_to_drop, axis=1)

data.head()